<a href="https://colab.research.google.com/github/abhinav96/js-weird-parts/blob/master/JS_weird_parts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JavaScript: Understanding The Weird Parts
This is a summary of my learnings from the course [JavaScript: Understanding The Weird Parts](https://www.udemy.com/course/understand-javascript/) taught by Anthony Alicea

Here's a link to my certificate of completion: https://ude.my/UC-DJT4BFIO

This notebook is not meant as a guide for JS or an alternative to the course; it is just a quick look of some important concepts of JS.

The course is a great way to understand core concepts necessary for working with JavaScript.

## Initializing Notebook Runtime in Google Colab

This notebook was written in Google Colab. The following is necessary to setup JavaScript kernel in that environment.

Skip this if the notebook is running in another environment where the kernel is already setup.

Credit: https://stackoverflow.com/a/60507269

In [ ]:
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global

## Execution context and exection stack

In JavaScript, all code is run inside an execution context.
An execution context is just a wrapper created by the JS engine, inside which your code is run.

### Global object and ```this```
The JS engine creates 2 things for you in this wrapper:
1. Global object (```global``` or ```window``` in case of browsers)
2. ```this``` variable

In [3]:
global

{ global: [Circular],
  process: 
   process {
     title: '',
     version: 'v8.11.3',
     moduleLoadList: 
      [ 'Binding contextify',
        'Binding natives',
        'Binding config',
        'NativeModule events',
        'Binding async_wrap',
        'Binding icu',
        'NativeModule util',
        'NativeModule internal/errors',
        'NativeModule internal/encoding',
        'NativeModule internal/util',
        'Binding util',
        'Binding constants',
        'NativeModule internal/util/types',
        'Binding buffer',
        'NativeModule buffer',
        'NativeModule internal/buffer',
        'Binding uv',
        'NativeModule internal/process',
        'NativeModule internal/process/warning',
        'NativeModule internal/process/next_tick',
        'NativeModule internal/async_hooks',
        'NativeModule internal/process/promises',
        'NativeModule internal/process/stdio',
        'Binding performance',
        'NativeModule perf_hooks',
        '

You may notice that ```global``` and ```this``` are the same object.

In [4]:
global === this;

true

But that is because the current line of code is running inside the global execution context.

The value of ```this``` changes based on how the code is executed. Because the above code is running in the global execution context, ```this``` points to ```global```.

**```this``` does not always point to ```global```.** (More on this later)

### Variables are attaced to the execution context

Any variables that are created outside of a function are attached to the ```global``` object.

In [5]:
var myVar = 'This variable is attached to the global object';
global.myVar;

'This variable is attached to the global object'

### Invoking functions create their own execution context

Whenever a function is called, a new execution context is created.
Any variables created inside the function are attached to the function's context, keeping other execution contexts (like global) unaffected.

In [6]:
var globalVar = 'This variable is attached to the global object';
global.globalVar;

'This variable is attached to the global object'

In [7]:
function myFunction() {
  var functionVar = "This variable is attached to the function's context and is only accessible inside the function";
  console.log(functionVar);
  console.log(global.functionVar); // undefined, because functionVar does not affect the global context.
}
myFunction();

This variable is attached to the function's context and is only accessible inside the function
undefined


In [8]:
try {
  console.log(functionVar); // also undefined, because the variable is not accessible outside the function's execution context
} catch (e) {
  console.log("Error is thrown when accessing an undefined variable in strict mode");
}

Error is thrown when accessing an undefined variable in strict mode


### Execution stack

As seen above, _function invocation_ creates a new execution context.
So if we invoke a function ```a``` which invokes another function ```b```, we'll have 3 execution contexts:

1. The global execution context
2. Execution context of ```a```
3. Execution context of ```b```

And these are structured like a stack where global is at the bottom, ```a``` sits above it, and ```b``` is at the top.

global => a => b (top of stack)

**This concept is similar to stack space in other languages**

## Scope chain and lexical environment

### The scope chain

You can access variables defined on the global execution context while being inside a function, like the following

In [9]:
var outsideVar = 'this variable which is defined outside of the function';
function myFunction() {
  console.log("I can access " + outsideVar);
}
myFunction();

I can access this variable which is defined outside of the function


You can also access variables declared in an outer function, like the following:

In [10]:
function a() {
  var closureVar = "this variable defined in the outer function";
  function innerFunction() {
    console.log("I can also access " + closureVar);
  }
  innerFunction();
}
a();

I can also access this variable defined in the outer function


In general, you can access any variables which are in the current scope chain.

#### Defining the scope chain
When a variable is not available in the current execution context, JS engines looks for the variable in the parent execution context. This happens recursively, until we reach the global execution context, which does not have a parent context.

This chain of execution contexts is also known as the **scope chain**

### Scope chain !== execution stack

In [11]:
function a() {
  var avar = 1;
  b();
}
function b() {
  console.log(avar); // won't work
}
a();

ReferenceError: ignored

It might be tempting to think that: because function ```a``` called function ```b```, any variable declared inside ```a``` will be accessible in ```b```.
But that's not true.

Even though in the execution stack, ```a```'s contex is right below ```b```'s context, ```a``` is not in the scope chain of ```b```.

Infact ```b```'s parent scope is the global execution context.

### Scope chain === lexical environment.

#### Defining lexical environment

Lexical environment is the physical place where a piece of code is written.

In [12]:
function baseLevelFunction() {
  // baseLevelFunction's lexical environment is global.
  // Because it is written on the top level, not inside anything else
}

function myFunc() {
  function innerFunction() {
    // innerFunction's lexical environment is 'myFunc' because it is written inside 'myFunc'
  }

  // and myFunc's lexical environment is global. Because it is written on the top level.
}

#### You can access any variable in your lexical environment (read: scope chain)

In [13]:
var inGlobalScope = 1;
function baseLevelFunction() {
  console.log("I can access inGlobalScope: " + inGlobalScope);
  console.log("Because my code is written in the global lexical environment");
}
baseLevelFunction();

I can access inGlobalScope: 1
Because my code is written in the global lexical environment


In [14]:
var inGlobalScope = 1;
function myFunc() {
  console.log("Similarly, myFunc can also access variables in global scope, because its lexical environment is global. See, inGlobalScope: " + inGlobalScope);
  function innerFunction() {
    console.log("And innerFunction can also access inGlobalScope: " + inGlobalScope);
    console.log("by traversing the scope chain, innerFunction => myFunc => global");
  }
  innerFunction();
}
myFunc();

Similarly, myFunc can also access variables in global scope, because its lexical environment is global. See, inGlobalScope: 1
And innerFunction can also access inGlobalScope: 1
by traversing the scope chain, innerFunction => myFunc => global


## Type coercion and ```==``` vs ```===```

In [15]:
var str = '1';
var num = 1;

In [16]:
console.log(str == num);
console.log(str === num);

true
false


When using ```==```, both the sides are **coerced** to the same type, and then compared. In the above example, the string '1' is converted to number 1, so both types match, and then compared if they are equal.

When using ```===```, the types are not coerced, both the types and the values must be equal to match.

You can see which type is converted to which other type here:
https://developer.mozilla.org/en-US/docs/Web/JavaScript/Equality_comparisons_and_sameness#Loose_equality_using

The summary is:

**The comparison x == y, where x and y are values, produces true or false. Such a comparison is performed as follows:**

1. If Type(x) is the same as Type(y), then return the result of performing Strict Equality Comparison x === y.
2. If x is null and y is undefined, return true.
3. If x is undefined and y is null, return true.
4. If Type(x) is Number and Type(y) is String, return the result of the comparison x == ToNumber(y).
5. If Type(x) is String and Type(y) is Number, return the result of the comparison ToNumber(x) == y.
6. If Type(x) is Boolean, return the result of the comparison ToNumber(x) == y.
7. If Type(y) is Boolean, return the result of the comparison x == ToNumber(y).
8. If Type(x) is either String, Number, or Symbol and Type(y) is Object, then return the result of the comparison x == ToPrimitive(y).
9. If Type(x) is Object and Type(y) is either String, Number, or Symbol, then return the result of the comparison ToPrimitive(x) == y.
10. Return false.

Credit: https://stackoverflow.com/a/54567595

## Functions are objects

Functions are just objects of type 'Function'. 

Just like objects, functions have properties and methods attached to them.

For example, functions have a ```name``` property and a ```toString``` method.

In [17]:
function anObject() {
  return 42;
};
console.log(anObject.name);
console.log(anObject.toString());

anObject
function anObject() {
  return 42;
}


#### You can do all operations on functions which are possible with objects.

Hold the function in a variable, just like an object

In [18]:
var funcInVariable = function () {
  console.log("I am a function");
}

Take a function as a parameter, just like an object

In [19]:
function iTakeAFunction(func) {
  func(); // and then execute it
}
iTakeAFunction(funcInVariable);

I am a function


Return a function, like an object

In [20]:
function iReturnAFunction(name) {
  return function () {
    console.log("Hello, " + name);
  }
}
var greetTheWorld = iReturnAFunction("World");
greetTheWorld(); // execute a function returned by a function

Hello, World


## ```this``` is not what you expect

Unlike other languages like Java where ```this``` refers to the current object; In JavaScript ```this``` does not always refer to the current object.

```this``` depends on how you **invoke** the function.

In [21]:
function giveMeThis() {
  return this;
}

// 1. when executed directly, this refers to window/global
console.log(giveMeThis() === global);

// 2. when executed using dot operator on an object, this refers to the object.
var obj = {
  attachedFunc: giveMeThis
};
console.log(obj.attachedFunc() === obj);

// irrespective of how the function was attached to the object.
obj.anotherAttachedFunc = giveMeThis;
console.log(obj.anotherAttachedFunc() === obj);

// [1] is true irrespective of where the function was declared/defined.
var f = obj.attachedFunc;
console.log(f() === global);

true
true
true
true


### The inner function caveat

When invoking an inner-function, you would think that the value of ```this``` won't change 

In [22]:
var myObj = {
  parentFunction: function () {
    console.log("parentFunction's this is myObj: " + (this === myObj));

    function innerFunction() {
      console.log("but innerFunction's this is not myObj: " + (this === myObj));
    }

    innerFunction(); // because you are invoking the function directly
  }
}

myObj.parentFunction();

parentFunction's this is myObj: true
but innerFunction's this is not myObj: false


#### The ```self``` variable

A common pattern in JS to solve the above problem is to declare a variable named ```self``` and use that everywhere instead of ```this```.

In [23]:
var myObj = {
  parentFunction: function () {
    var self = this;

    function innerFunction() {
      console.log('now self is myObj: ' + (self === myObj)); // using lexical env
    }

    innerFunction();
  }
}

myObj.parentFunction();

now self is myObj: true


### ```this``` refers to the new object when calling a function constructor using ```new``` operator

In [24]:
function Person(firstName, lastName) { // a constructor function
  this.firstName = firstName; // 'this' refers to the newly created object
  this.lastName = lastName;
}

var person = new Person('name', 'surname');
console.log(person);

Person { firstName: 'name', lastName: 'surname' }


## Fixating ```this``` (```call```, ```apply``` and ```bind```)

### ```call```

Every function has a ```call``` property, which is also a function. (remember, functions are objects)

```call``` executes the function with ```this``` set as the first argument provided. All arguments passed to ```call``` after the first one are passed to the function.

In [25]:
function printThis(arg) {
  console.log("this === global? : " + (this === global));
  if (this !== global) {
    console.log(this);
  }
  console.log('called with ' + arg);
}

When invoked normally, ```this``` refers to ```global```

In [26]:
printThis(1);

this === global? : true
called with 1


When invoked using ```call```, ```this``` refers to the 1st argument

In [27]:
var obj = {
  key: 'just another object'
}
printThis.call(obj, 2);

this === global? : false
{ key: 'just another object' }
called with 2


### ```apply```

```apply``` works same as ```call``` but it takes arguments as an array.

In [28]:
printThis.apply(obj, [3]);

this === global? : false
{ key: 'just another object' }
called with 3


### ```bind```

```bind``` creates a new function which has the ```this``` fixed to what you want, irrespective of how you call the function later.

In [29]:
var boundThis = printThis.bind(obj);
boundThis(4); // if bind wasn't used, this ivocation would make this == global

this === global? : false
{ key: 'just another object' }
called with 4


## IIFE – Immediately Invoked Function Expressions

You can define a function and execute it immediately. This is called an IIFE.

In [30]:
var output = function (num) {
  return num + 5;
}(1); // notice the "(1)"

output;

6

### Standalone IIFE

In [31]:
function () {
  // you cannot write an anonymous function like this 
} ();

function name() {

} (); // and you cannot execute a named function directly

SyntaxError: ignored

But you can put the function in parenthesis to convert it into an expression

In [32]:
( // an expression that returns a function
  function () {
    console.log('I am an IIFE');
  }
)
(); // execute the function returned by the previous expression

I am an IIFE


In [33]:
( // convert to expression
  function name() { // named function
    console.log('named IIFE');
  }() // execute it
);

named IIFE


### Why use IIFE?
IIFEs help create a separate namespace. 

Because function invocation creates a new execution context, it ensures that anything declared inside the function does not collide with the anything outside (like the global execution context).


## Closures

A closure is the combination of a function bundled together (enclosed) with references to its surrounding state (the lexical environment). In other words, a closure gives you access to an outer function’s scope from an inner function.

https://developer.mozilla.org/en-US/docs/Web/JavaScript/Closures

In [34]:
function createPrintNameFunction(name) {
  return function () {
    console.log('Hello ' + name); // notice, name was passed to the outer function.
  }
} // and the outer function has completed its execution

var printTony = createPrintNameFunction('Tony');
var printStark = createPrintNameFunction('Stark');

// but even when calling the inner function at a later time,
// we have access to variables of the outer function
printTony();
printStark();
// now re-read the definition

Hello Tony
Hello Stark


### Variables in the closure can change after the inner function is created.

In [35]:
var funcArr = [];
for (var i = 0; i < 3; ++i) {
  funcArr.push(
    function () {
      console.log(i);
    }
  )
}
funcArr[0]();
funcArr[1]();
funcArr[2]();

3
3
3


All functions print the same value, because all of them share the same closure, and the value of ```i``` is being updated in there.

We can fix this by creating a new closure for each of the function.

In [36]:
var funcArr = [];
for (var i = 0; i < 3; ++i) {
  funcArr.push(
    (function (j) { // make a new closure for each function
      return function () {
        console.log(j);
      }
    }(i)) // notice, IIFE
  )
}
funcArr[0]();
funcArr[1]();
funcArr[2]();

0
1
2


## Prototypical Inheritance

### The prototype chain

Every object has a prototype

In [37]:
var person = {
  name: 'Tony Stark'
};
person.__proto__; // DO NOT DO THIS IN PRODUCTION CODE

{}

The prototype can be thought of as a parent object of the current object. Any property which is availabe on the parent object is also available on the child object.

In [38]:
var person = {
  name: 'Tony Stark'
};
var employee = {
  empId: 10
};
employee.__proto__ = person; // DO NOT DO THIS IN PRODUCTION CODE
employee.name; // name is not set on 'employee' but available on its prototype, i.e. 'person'

'Tony Stark'

The prototype object can have a prototype of its own. This recursive chain of prototypes is known as the **prototype chain**

### Setting prototypes of newly created objects

#### The ```new``` operator and Function Constructors

The ```new``` operator creates a new (empty) object and then calls the function for initializing the newly created object. The initialization function is called a Function Constructor.
The Function Constructor can access the newly created object using ```this```.

In [39]:
function Person(firstName, lastName) { // Function Constructor
  console.log('I have received an empty new object: ');
  console.log(this);
  this.firstName = firstName;
  this.lastName = lastName;
  console.log('And I added 2 properties to it');
  console.log(this);
}

var tony = new Person('Tony', 'Stark');
tony;

I have received an empty new object: 
Person {}
And I added 2 properties to it
Person { firstName: 'Tony', lastName: 'Stark' }


Person { firstName: 'Tony', lastName: 'Stark' }

#### ```prototype``` defines ```__proto__```

1. Every **function** has a ```prototype``` property. But this is not the one which is in the prototype chain we saw earlier, that was ```__proto__``` and this is ```prototype```.

2. Only functions have a ```prototype``` property. Any other objects don't have it by default.

3. Any object created by a function constructor has its ```__proto__``` pointing to the constructor's ```prototype```. So whatever is set in the function's ```prototype``` is available in the object's prototype chain.

In [40]:
function Person(firstName, lastName) {
  this.firstName = firstName;
  this.lastName = lastName;
}
Person.prototype.getFullName = function () {
  return this.firstName + ' ' + this.lastName;
}

var tony = new Person('Tony', 'Stark');
console.log(tony.__proto__ === Person.prototype);
console.log(tony.getFullName());

var bruce = new Person('Bruce', 'Wayne');
// all objects created via the constructor have same prototype
console.log(bruce.__proto__ === tony.__proto__);
bruce.getFullName();

true
Tony Stark
true


'Bruce Wayne'

### Building a hierarchy

In [41]:
function Person(firstName, lastName) {
  this.firstName = firstName;
  this.lastName = lastName;
}
Person.prototype.getFullName = function () {
  return this.firstName + ' ' + this.lastName;
}

function SuperHero(firstName, lastName, superHeroName) {
  // do all initialization in the Person constructor
  Person.call(this, firstName, lastName); // notice, we fixated 'this' to the new object
  this.superHeroName = superHeroName;
}
// by default the prototype of SuperHero will be empty
console.log(SuperHero.prototype);
// but we want to inherit everything defined on "Person"'s prototype
// so we initialize the prototype with a new Person object
SuperHero.prototype = new Person();
console.log(SuperHero.prototype);

// now we can add anything extra for SuperHero objects
SuperHero.prototype.getSuperHeroName = function () {
  return this.superHeroName;
}

var ironMan = new SuperHero('Tony', 'Stark', 'Iron Man');
console.log(ironMan.getFullName()); // we have properties from Person
console.log(ironMan.getSuperHeroName()); // and also from SuperHero

SuperHero {}
Person { firstName: undefined, lastName: undefined }
Tony Stark
Iron Man


## ES6 features

The following link is a collection of the new features introduced in ES6 along with examples: http://git.io/es6features

### ```let``` and ```const```

Pitfalls of ```var```
1. ```var```'s scope is the closest enclosing function.
2. Redeclaring a ```var``` does not throw any errors.
3. ```var```s can still be accessed before declaring them

See:

In [42]:
function problemsWithVar(a) {
  console.log(count); // no error even though "count" is not yet defined. the value is undefined
  var count = 5;
  if (a) {
    var count = 25; // re-declaring count again inside 'if' block changes the value of the outer count
  }
  console.log(count);
  var count = 100; // var was already declared above twice. this is 3rd time. No errors.
  console.log(count);
}
problemsWithVar(true);

undefined
25
100


All these problems are solved in ES6 with the newly introduced keyword ```let```

In [43]:
function problemsWithVar(a) {
  // console.log(count); // will throw an error
  let count = 5;
  if (a) {
    let count = 25; // re-declaring with 'let' inside the if block does not affect the outer 'count'
    // it creates a new variable limited to block's scope
    console.log(count);
  }
  console.log(count);
  // let count = 100; // Re-defining a let throws an error.
}
problemsWithVar(true);

25
5


```const``` is same as ```let```, with the added constraint that it cannot be redefined later.

**Their value can still be changed though**

In [44]:
function cannotRedefineConst() {
  let a = 1;
  a = 2; // allowed

  const b = 1;
  b = 2; // not allowed, because it is a const
}
cannotRedefineConst();

TypeError: ignored

In [45]:
function changeConstObjectsValue() {
  const obj = {
    name: 'A'
  }

  obj.name = 'B'; // allowed
}
changeConstObjectsValue();

### Arrow functions

Arrow functions are similar to lambdas in Python and Java.

Example anonymous function for comparasion

In [118]:
var sum = function (a, b) {
  return a + b;
};
sum(1, 2);

3

Arrow function example

In [120]:
var diff = (a, b) => a - b; // implicit return for single line expressions
diff(1, 2);

-1

Parenthesis are optional for single param function

In [122]:
var square = a => a * a;
square(2);

4

Multi-line functions need braces and explicit return

In [128]:
var both = (a, b) => { // use braces for multi-line functions
  var sum = a + b;
  var diff = a - b;
  return { // explicit return is needed for multi-line functions
    sum: sum,
    diff: diff
  };
};
both(1, 2);

{ sum: 3, diff: -1 }

### Classes

The following code builds the same hierarchy as built previously using function constructors, but instead uses ES6's classes

In [47]:
class Person2 {
  constructor(firstName, lastName) {
    this.firstName = firstName;
    this.lastName = lastName;
  }

  getFullName() {
    return this.firstName + ' ' + this.lastName;
  }
}

class SuperHero2 extends Person2 {
  constructor(firstName, lastName, superHeroName) {
    super(firstName, lastName);
    this.superHeroName = superHeroName;
  }

  getSuperHeroName() {
    return this.superHeroName;
  }
}

In [48]:
var cap = new SuperHero2('Steve', 'Rogers', 'Captain America');
console.log(cap.getFullName() + ' is ' + cap.getSuperHeroName());

Steve Rogers is Captain America
